To have a more varied dataset we add entries for 2024 and change some of the cost centers to have multiple cost centers per business partner. Furthermore, this script checks what mappings are missing

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df_enriched = pd.read_csv('../data/export_co2_km.csv')

In [3]:
df_bp = pd.read_csv('../data/BP.csv')

In [4]:
df = pd.read_parquet('../data/anonymized_sap_data.parquet')

In [19]:
df_bp.shape

(3182, 15)

In [6]:
missing_von = set(df['Reise von'].dropna().unique()).difference(df_bp['offizielle_bezeichnung'])

def find_closest_station(entry, station_list):
    # If entry contains a comma, take the part before the comma for matching
    base = entry.split(',')[0].strip()
    # Filter to stations without a comma in their name
    candidates = [s for s in station_list if ',' not in s]
    # Find candidates that contain the base as substring (case-insensitive)
    matches = [s for s in candidates if base.lower() in s.lower()]
    # Return the first match if found, else None
    if matches is None:
        asdf = base.split(' ')[0].strip()
        candidates = [s for s in station_list if ',' not in s]
        # Find candidates that contain the base as substring (case-insensitive)
        matches = [s for s in candidates if asdf.lower() in s.lower()]
    return min(matches, key=len) if matches else None

station_list = df_bp['offizielle_bezeichnung'].tolist()
closest_matches = {entry: find_closest_station(entry, station_list) for entry in missing_von}
closest_matches = {k: v for k, v in closest_matches.items() if v is not None}


In [10]:
new_entries = []
for key, value in closest_matches.items():
    # Find the row in df_bp where 'offizielle_bezeichnung' == value
    row = df_bp.loc[df_bp['offizielle_bezeichnung'] == value][['dienststellen_id', 'abkuerzung',  'bfs_nummer', 'verkehrsmittel_abkuerzung', 'verkehrsmittel_de', 'verkehrsmittel_fr', 'verkehrsmittel_it', 'go_abkuerzung_de', 'go_abkuerzung_fr', 'go_abkuerzung_it','e_lv95','n_lv95','e_wgs84','n_wgs84']]
    if not row.empty:
        new_entries.append([key] + row.values.flatten().tolist())

In [11]:
new_entries_df = pd.DataFrame(new_entries, columns=['offizielle_bezeichnung', 'dienststellen_id', 'abkuerzung',  'bfs_nummer', 'verkehrsmittel_abkuerzung', 'verkehrsmittel_de', 'verkehrsmittel_fr', 'verkehrsmittel_it', 'go_abkuerzung_de', 'go_abkuerzung_fr', 'go_abkuerzung_it','e_lv95','n_lv95','e_wgs84','n_wgs84'])

In [12]:
all_available_bps = pd.concat([df_bp, new_entries_df], ignore_index=True)

In [16]:
all_available_bps.to_csv('../data/BP_complete.csv', index=False)

In [17]:
x= df.groupby('Vertragskonto').size().reset_index(name='Anzahl Zeilen')

px.histogram(x, x='Anzahl Zeilen')

Vertragskonto    9999925.0
Anzahl Zeilen      44857.0
dtype: float64

In [ ]:
x.sort_values('Anzahl Zeilen', ascending=False).head(20)

In [ ]:
df.groupby('Geschäftsfall ID').size().sort_values(ascending=False)

In [ ]:
df.groupby('Geschäftspartner')['Vertragskonto'].unique().map(len).max()

In [ ]:
df.groupby('Vertragskonto')['Geschäftspartner'].unique().map(len).max()

In [ ]:
df[['Reise von', 'Reise nach']].drop_duplicates()

In [ ]:
df['Reise von']

In [ ]:
df['Reise nach'].unique()

In [ ]:
df.columns

In [ ]:
df['Hinjahr'] = df['Hinreisedatum'].dt.year
df['Rückjahr'] = df['Rückreisedatum'].dt.year

df